# Fine-Tune Amharic NER Model
This notebook fine-tunes a model like AfroXLMR or BERT for NER on Amharic Telegram product data.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets seqeval accelerate pandas evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.9 MB/s eta 0:00:00
  

In [ ]:
# 📦 Install all dependencies (run only once)
!pip install transformers datasets seqeval evaluate -q



# 🧼 Parse CoNLL .txt file
lines = open("/content/drive/MyDrive/labeled_telegram_product_price_location.txt", "r", encoding="utf-8").read().strip().split("\n")
sentences, tokens, labels = [], [], []
for line in lines:
    if line.strip() == "":
        if tokens:
            sentences.append((tokens, labels))
            tokens, labels = [], []
    else:
        token, label = line.split()
        tokens.append(token)
        labels.append(label)
if tokens:
    sentences.append((tokens, labels))

# 🔄 Convert to Hugging Face Dataset
from datasets import Dataset
label_list = sorted(list({l for _, labs in sentences for l in labs}))
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}
hf_data = [{"tokens": t, "ner_tags": [label_to_id[l] for l in labs]} for t, labs in sentences]
dataset = Dataset.from_list(hf_data).train_test_split(test_size=0.1)

# 🔠 Tokenization + Label Alignment
from transformers import AutoTokenizer
checkpoint = "Davlan/bert-base-amharic"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_and_align(example):
    tokenized = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    word_ids = tokenized.word_ids()
    labels = []
    prev_word = None
    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        elif word_id != prev_word:
            labels.append(example["ner_tags"][word_id])
        else:
            labels.append(example["ner_tags"][word_id])
        prev_word = word_id
    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(tokenize_and_align)

# 🧠 Training Setup
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import evaluate
import os

# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"

model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=len(label_list))
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    true_preds = [[id_to_label[p] for p, l in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]
    true_labels = [[id_to_label[l] for p, l in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]
    return metric.compute(predictions=true_preds, references=true_labels)

# 🔧 TrainingArguments
args = TrainingArguments(
    output_dir="amharic-ner-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# 🚀 Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 💾 Save final model locally
trainer.save_model("amharic-ner-bert")
tokenizer.save_pretrained("amharic-ner-bert")


In [ ]:
# 📦 STEP 1: Install Dependencies
!pip install --upgrade transformers datasets seqeval evaluate -q

# 📂 STEP 2: Upload Your CoNLL File
from google.colab import files
uploaded = files.upload()  # Upload labeled_telegram_product_price_location.txt

# 🧼 STEP 3: Parse CoNLL Format
import pandas as pd

lines = open("/content/drive/MyDrive/labeled_telegram_product_price_location.txt", "r", encoding="utf-8").read().strip().split("\n")
sentences = []
tokens, labels = [], []

for line in lines:
    line = line.strip()
    if not line:
        if tokens:
            sentences.append((tokens, labels))
            tokens, labels = [], []
    else:
        parts = line.split()
        if len(parts) == 2:
            token, label = parts
            tokens.append(token)
            labels.append(label)
if tokens:
    sentences.append((tokens, labels))

# 📊 STEP 4: Prepare HF Dataset
from datasets import Dataset
from collections import defaultdict

label_list = sorted(list({l for _, labs in sentences for l in labs}))
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

hf_data = [{"tokens": t, "ner_tags": [label_to_id[l] for l in labs]} for t, labs in sentences]
dataset = Dataset.from_list(hf_data).train_test_split(test_size=0.1)

# 🔤 STEP 5: Tokenize and Align Labels
from transformers import AutoTokenizer
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    labels = []
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(example["ner_tags"][word_idx])
        else:
            labels.append(example["ner_tags"][word_idx])
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels)

# 🧠 STEP 6: Model Setup
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import evaluate

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    true_preds = [[id_to_label[p] for p, l in zip(pred, label) if l != -100]
                  for pred, label in zip(predictions, labels)]
    true_labels = [[id_to_label[l] for p, l in zip(pred, label) if l != -100]
                   for pred, label in zip(predictions, labels)]
    return metric.compute(predictions=true_preds, references=true_labels)

# ❌ Disable Weights & Biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

# 🎯 STEP 7: Training Arguments
training_args = TrainingArguments(
    output_dir="xlmr-amharic-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# 🚀 STEP 8: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 💾 STEP 9: Save Model
trainer.save_model("amharic-ner-xlmr")
tokenizer.save_pretrained("amharic-ner-xlmr")
